#### Steps -

1. Install tabula from this path: https://tabula.technology/
2. (optional) Install/get subscription for YNAB: https://www.youneedabudget.com/
3. Create folder path like so:
    \root\
    \root\<year>\1-tabula-output\
    \root\<year>\2-cleaned\
    \root\<year>\3-processed\
    \root\<year>\4-ynab\

4. Download pdf statement and put in \<year>\0-pdf\
5. Open tabula, and generate csv, paste in \<year>\1-tabula-output\
6. Open file from step 2, and clean up as follows
  - no header needed
  - 6 columns: date, transaction type, payee, outflow, inflow, balance
  - some columns may have shifted... adjust them
  - sometimes strange characters in first row etc... remove them
  - save file
7. Open \Balances.csv
  - Confirm date column is yyyy-mm-dd
  - Add new row for current month
  - Populate opening, closing balance from pdf
  - Drag-fill 'ThisMonth_Closing_NextMonth_Opening_Reconciled' for previous row and confirm it is TRUE
8. Run jupyter notebook
9. Populate 'Stmt_Balance_Notebook_Balance_Reconciled' if TRUE, if not investigate
10. Load file from \<year>\4-ynab\ into YNAB

In [19]:
import pandas as pd
import numpy as np
import copy
import os
%config Completer.use_jedi = False

In [20]:
root = 'D:\\MyDocuments\\Bank-Statements\\HSBC\\'
root_path = root + '2022\\'
tabula_path = root_path + '1-tabula-output\\'
cleaned_path = root_path + '2-cleaned\\'
processed_path = root_path + '3-processed\\'
ynab_path = root_path + '4-ynab\\'
balances_file = root + 'Balances.csv'

## Table of contents <a id="0"></a>
* [Generate list of files](#file-list)
* [Cleanup dataset](#cleanup)
* [Process dataset](#process)
* [Validate dataset](#validate)
* [Generate YNAB files](#ynab)


### Generate list of files <a name="file-list"></a>
[Go back to top](#0)

In [21]:
result = [{'statement_date': f[0:10], 'filename': f, 'original_full_path': os.path.join(dp, f)}
          for dp, dn, filenames in os.walk(tabula_path) 
          for f in filenames 
          if (os.path.splitext(f)[1] == '.csv') 
          and ('_processed' not in os.path.splitext(f)[0]) 
          and ('_ynab' not in os.path.splitext(f)[0])]


result

[{'statement_date': '2022-01-25',
  'filename': '2022-01-25_Statement.csv',
  'original_full_path': 'D:\\MyDocuments\\Bank-Statements\\HSBC\\2022\\1-tabula-output\\2022-01-25_Statement.csv'},
 {'statement_date': '2022-02-25',
  'filename': '2022-02-25_Statement.csv',
  'original_full_path': 'D:\\MyDocuments\\Bank-Statements\\HSBC\\2022\\1-tabula-output\\2022-02-25_Statement.csv'}]

In [22]:
df_balances = pd.read_csv(balances_file)
cols = ['stmt_date', 'opening', 'closing']
df_balances.columns = cols
df_balances = df_balances.astype({"stmt_date": 'datetime64'
                                  ,"opening": 'float64'
                                  ,"closing": 'float64'})
df_balances.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   stmt_date  92 non-null     datetime64[ns]
 1   opening    92 non-null     float64       
 2   closing    92 non-null     float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 2.3 KB


### Clean up dataset <a name="cleanup"></a>
[Go back to top](#0)

1. Add header rows
2. Add date in all rows
3. Combine multi-line payees
4. Remove 'Balance carried forward', 'Balance brought forward' rows within dataset

In [23]:
def clean_df(df_param):
    df = copy.deepcopy(df_param)
    header_cols = ['date', 'transaction_type', 'payee', 'outflow', 'inflow', 'balance']
    
    # set columns
    df.columns = header_cols
    
    # transform date
    df[['payee', 'outflow', 'inflow', 'balance']] = df[['payee', 'outflow', 'inflow', 'balance']].astype(str)
    df['date'] = pd.to_datetime(
        df['date'].str[:2] 
        + '-' 
        + df['date'].str[3:6] 
        + '-' 
        + '20' + df['date'].str[-2:]
    )
    
    # remove unneeded columns
    cols = ['date', 'payee', 'outflow', 'inflow', 'balance']
    df = df[cols]
    df.reset_index(drop=True, inplace=True)
    
    # change data types
    df.loc[:, 'balance'] = df['balance'].str.replace(',', '')
    df.loc[:, 'outflow'] = df['outflow'].str.replace(',', '')
    df.loc[:, 'inflow'] = df['inflow'].str.replace(',', '')
    df.loc[df['outflow'] == '.', 'outflow'] = 0.0
    df = df.astype({'outflow': float, 'inflow': float, 'balance': float})
    
    df['outflow'].fillna(0, inplace=True)
    df['inflow'].fillna(0, inplace=True)
    df['balance'].fillna(0, inplace=True)
    
    # remove balance carried forward rows within dataset
    df_temp = df.iloc[1:-1, :]
    idx = df_temp.loc[df_temp['payee'].str.contains('(?i)balance'), :].index
    df.drop(idx, inplace=True)
    
    # combine multi-line payees
    for idx, row in df.iterrows():
        if ((df.loc[idx, 'outflow']==0.0) and (df.loc[idx, 'inflow']==0.0) and (df.loc[idx, 'balance']==0.0)):
            df.loc[idx+1, 'payee'] = df.loc[idx, 'payee'] + ' ' + df.loc[idx+1, 'payee']
            df.loc[idx+1, 'date'] = df.loc[idx, 'date']
    
    df.drop(df[(df['outflow']==0.0) & (df['inflow']==0.0) & (df['balance']==0.0)].index, inplace=True)
    df['date'].fillna(method='ffill', inplace=True)
    
    return df

In [24]:
# clean up files
for file in result:
    original_full_path = file['original_full_path']
    print('Cleaning up file: [{filename}]'.format(filename=original_full_path))
    df_in = pd.read_csv(original_full_path, header=None)
    df = clean_df(df_in)
    file_name = os.path.basename(original_full_path)
    cleaned_df_filename = os.path.splitext(file_name)[0] + '_cleaned.csv'
    cleaned_df_full_path = os.path.join(cleaned_path, cleaned_df_filename)
    file['cleaned_full_path'] = cleaned_df_full_path
    df.to_csv(cleaned_df_full_path, index=False)
    print('Cleaning up complete: [{filename}]'.format(filename=cleaned_df_full_path))

cleaned_result = [os.path.join(dp, f)
                  for dp, dn, filenames in os.walk(cleaned_path) 
                  for f in filenames 
                  if (os.path.splitext(f)[1] == '.csv')]

Cleaning up file: [D:\MyDocuments\Bank-Statements\HSBC\2022\1-tabula-output\2022-01-25_Statement.csv]
Cleaning up complete: [D:\MyDocuments\Bank-Statements\HSBC\2022\2-cleaned\2022-01-25_Statement_cleaned.csv]
Cleaning up file: [D:\MyDocuments\Bank-Statements\HSBC\2022\1-tabula-output\2022-02-25_Statement.csv]
Cleaning up complete: [D:\MyDocuments\Bank-Statements\HSBC\2022\2-cleaned\2022-02-25_Statement_cleaned.csv]


### Process cleaned up files <a name="process"></a>
[Go back to top](#0)

Combine outflow and inflow amount

In [25]:
def process_df(df_param):
    df = copy.deepcopy(df_param)
    
    # combine outflow and inflow into single amount column
    df.loc[:, 'outflow'] = (df['outflow']*-1) + df['inflow']
    df = df.rename(columns={'outflow': 'amount'})
    df.drop(df[(df['amount']==0.0) & (df['balance']==0.0)].index, inplace=True)
    df['date'].fillna(method='ffill', inplace=True)
    
    cols = ['date', 'payee', 'amount', 'balance']
    

    df = df[cols]
    
    return df    

In [26]:
# process files
for file in result:
    cleaned_full_path = file['cleaned_full_path']
    
    print('Processing file: [{filename}]'.format(filename=cleaned_full_path))
    df_in = pd.read_csv(cleaned_full_path)
    df = process_df(df_in)
    file_name = os.path.basename(cleaned_full_path)
    processed_df_filename = os.path.splitext(file_name)[0] + '_processed.csv'
    processed_df_full_path = os.path.join(processed_path, processed_df_filename)
    
    file['processed_full_path'] = processed_df_full_path
    
    df.to_csv(processed_df_full_path, index=False)
    print('Processing complete: [{filename}]'.format(filename=processed_df_full_path))

processed_result = [os.path.join(dp, f)
                  for dp, dn, filenames in os.walk(processed_path) 
                  for f in filenames 
                  if (os.path.splitext(f)[1] == '.csv')]

Processing file: [D:\MyDocuments\Bank-Statements\HSBC\2022\2-cleaned\2022-01-25_Statement_cleaned.csv]
Processing complete: [D:\MyDocuments\Bank-Statements\HSBC\2022\3-processed\2022-01-25_Statement_cleaned_processed.csv]
Processing file: [D:\MyDocuments\Bank-Statements\HSBC\2022\2-cleaned\2022-02-25_Statement_cleaned.csv]
Processing complete: [D:\MyDocuments\Bank-Statements\HSBC\2022\3-processed\2022-02-25_Statement_cleaned_processed.csv]


### Validate processed files <a name="validate"></a>
[Go back to top](#0)

Validate opening and closing balance amount with statements

In [27]:
def validate_df(df_param):
    df = copy.deepcopy(df_param)
    
    # validate transactions, by comparing balances
    opening_balance = df.loc[df['payee'].str.upper() == 'BALANCE BROUGHT FORWARD', 'balance'].values[0]
    
    df = df.loc[(df['payee'].str.upper() != 'BALANCE BROUGHT FORWARD') 
                & (df['payee'].str.upper() != 'BALANCE CARRIED FORWARD'), :]
    
    df_datewise = df.groupby('date').sum(['amount', 'balance'])
    df_datewise.reset_index(inplace=True)
    for idx, row in df_datewise.iterrows():
        if idx <= len(df_datewise.index) - 2: 
            df_datewise.loc[idx+1, 'calculated_balance'] = df_datewise.loc[idx, 'balance'] + df_datewise.loc[idx+1, 'amount']
    
    closing_balance = df_datewise.loc[len(df_datewise.index)-1, 'calculated_balance']
    
    return opening_balance, closing_balance

In [28]:
# validate files
for file in result:
    processed_full_path = file['processed_full_path']
    
    print('Validating file: [{filename}]'.format(filename=processed_full_path))
    df_in = pd.read_csv(processed_full_path)
    opening_balance, closing_balance = validate_df(df_in)
    
    file['opening'] = opening_balance
    file['closing'] = closing_balance

Validating file: [D:\MyDocuments\Bank-Statements\HSBC\2022\3-processed\2022-01-25_Statement_cleaned_processed.csv]
Validating file: [D:\MyDocuments\Bank-Statements\HSBC\2022\3-processed\2022-02-25_Statement_cleaned_processed.csv]


In [29]:
df_balances.head()

,stmt_date,opening,closing
0,2014-07-25,21274.59,23052.24
1,2014-08-25,23052.24,24671.14
2,2014-09-25,24671.14,26486.91
3,2014-10-25,26486.91,28131.31
4,2014-11-25,28131.31,26351.06


In [30]:
df_result = pd.DataFrame.from_records(result)
df_result

,statement_date,filename,original_full_path,cleaned_full_path,processed_full_path,opening,closing
0,2022-01-25,2022-01-25_Statement.csv,D:\MyDocuments\Bank-Statements\HSBC\2022\1-tab...,D:\MyDocuments\Bank-Statements\HSBC\2022\2-cle...,D:\MyDocuments\Bank-Statements\HSBC\2022\3-pro...,9646.72,11383.30
1,2022-02-25,2022-02-25_Statement.csv,D:\MyDocuments\Bank-Statements\HSBC\2022\1-tab...,D:\MyDocuments\Bank-Statements\HSBC\2022\2-cle...,D:\MyDocuments\Bank-Statements\HSBC\2022\3-pro...,11383.30,12532.55


In [31]:
cols = ['statement_date', 'opening', 'closing']
df_result = df_result[cols]
cols = ['stmt_date', 'opening', 'closing']
df_result.columns = cols
df_result = df_result.astype({'stmt_date': 'datetime64'})
df_result = pd.merge(df_result, df_balances, how='left', on='stmt_date', suffixes=['_calc', '_stmt'])
df_result['prev_closing'] = df_result['closing_calc'].shift(1)

df_result['balances_valid'] = pd.Series((round(df_result['opening_calc'], 2) == round(df_result['opening_stmt'], 2)) 
                               & (round(df_result['closing_calc'], 2) == round(df_result['closing_stmt'], 2)))

df_result['opening_valid'] = pd.Series((round(df_result['prev_closing'], 2) == round(df_result['opening_calc'], 2)))

df_result

,stmt_date,opening_calc,closing_calc,opening_stmt,closing_stmt,prev_closing,balances_valid,opening_valid
0,2022-01-25,9646.72,11383.30,9646.72,11383.3,NaN,True,False
1,2022-02-25,11383.30,12532.55,NaN,NaN,11383.3,False,True


### Genarate YNAB files <a name="ynab"></a>
[Go back to top](#0)

In [32]:
def transform_payee(in_str: str):
    out_str = in_str
    category = ""
    mapping = [{"str": 'asda', 'repl': 'Asda', 'category': 'Monthly recurring: Groceries'}
               , {"str": 'tesco', 'repl': 'Tesco', 'category': 'Monthly recurring: Groceries'}
               , {"str": 'sainsburys', 'repl': 'Sainsburys', 'category': 'Monthly recurring: Groceries'}
               , {"str": 'morrisons', 'repl': 'Morrisons', 'category': 'Monthly recurring: Groceries'}
               , {"str": 'taj stores', 'repl': 'Taj Stores', 'category': 'Monthly recurring: Groceries'}
               , {"str": 'waitrose', 'repl': 'Waitrose', 'category': 'Monthly recurring: Groceries'}
               
               , {"str": 'ee limited', 'repl': 'EE Limited', 'category': 'Monthly recurring: Phone'}
               , {"str": 'the gym ltd', 'repl': 'The Gym Ltd', 'category': 'Monthly recurring: Gym'}
               , {"str": 'lul ticket', 'repl': 'TFL', 'category': 'Monthly recurring: Transport'}
               , {"str": 'oyster', 'repl': 'TFL', 'category': 'Monthly recurring: Transport'}
               , {"str": 'igloo energy', 'repl': 'Igloo', 'category': 'Monthly recurring: Gas and electric'}
               , {"str": 'virgin media', 'repl': 'Virgin Media', 'category': 'Monthly recurring: Internet'}
               
               , {"str": 'dominos', 'repl': 'Dominos', 'category': 'Discretionary: Entertainment'}
               , {"str": 'papa johns', 'repl': 'Papa Johns', 'category': 'Discretionary: Entertainment'}
               , {"str": 'just-eat', 'repl': 'Just Eat', 'category': 'Discretionary: Entertainment'}
               , {"str": 'justeat', 'repl': 'Just Eat', 'category': 'Discretionary: Entertainment'}
               , {"str": 'just eat', 'repl': 'Just Eat', 'category': 'Discretionary: Entertainment'}
               , {"str": 'feast burgers', 'repl': 'Feast Burgers', 'category': 'Discretionary: Entertainment'}
               , {"str": 'deliveroo', 'repl': 'Deliveroo', 'category': 'Discretionary: Entertainment'}
               , {"str": 'mcdonalds', 'repl': 'McDonald''s', 'category': 'Discretionary: Entertainment'}
               
               , {"str": 'netflix', 'repl': 'Netflix', 'category': 'Monthly: Streaming'}
               , {"str": 'amazonprime', 'repl': 'Amazon Prime', 'category': 'Monthly: Streaming'}
               , {"str": 'amazon prime', 'repl': 'Amazon Prime', 'category': 'Monthly: Streaming'}
               
               , {"str": 'primark', 'repl': 'Primark', 'category': 'Discretionary: Misc Purchases'}
               
               , {"str": 's j p fish', 'repl': in_str, 'category': 'Basics: Rent'}
               
               , {"str": 'farrow-ball.com', 'repl': 'Farrow and ball', 'category': ''}
               , {"str": 'argos', 'repl': 'Argos', 'category': ''}
               , {"str": 'amazon.co.uk', 'repl': 'Amazon', 'category': ''}
               , {"str": 'uber*trip', 'repl': 'Uber', 'category': ''}
               , {"str": 'uber *trip', 'repl': 'Uber', 'category': ''}
               , {"str": 'poundland', 'repl': 'Poundland', 'category': ''}
               , {"str": 'ikea', 'repl': 'Ikea', 'category': ''}
               , {"str": 'johnlewis', 'repl': 'John Lewis', 'category': ''}
               , {"str": 'john lewis', 'repl': 'John Lewis', 'category': ''}
               , {"str": 'independent contra emp', 'repl': 'ICS', 'category': ''}
               , {"str": 'barclays', 'repl': 'Barclays', 'category': ''}
              ]
    for search_string in mapping:
        if search_string['str'] in in_str.lower():
            out_str = search_string['repl']
            category = search_string['category']
        
    if 'pizza' in in_str.lower() and 'hut' in in_str.lower():
        out_str = 'Pizza Hut'
        category = 'Discretionary: Entertainment'
    elif 'papa' in in_str.lower() and 'john' in in_str.lower():
        out_str = 'Papa Johns'
        category = 'Discretionary: Entertainment'
        
    return out_str, category

In [33]:
def process_ynab(df_param):
    df_ynab = copy.deepcopy(df_param)
    
    # rename columns
    df_ynab = df_ynab.rename(columns={'date': 'Date', 'payee': 'Payee'})
    df_ynab['Date'] = pd.to_datetime(df_ynab['Date'])
    
    # remove balance rows
    idx = df_ynab.loc[df_ynab['Payee'].str.contains('(?i)balance'), :].index
    df_ynab.drop(idx, inplace=True)
    
    # create additional ynab columns, format values
    df_ynab['Category'] = ''
    df_ynab['Memo'] = ''
    df_ynab.loc[df_ynab['amount'] < 0, 'Outflow'] = df_ynab['amount'] * -1
    df_ynab.loc[df_ynab['amount'] > 0, 'Inflow'] = df_ynab['amount']
    df_ynab['Outflow'].fillna('', inplace=True)
    df_ynab['Inflow'].fillna('', inplace=True)
    cols = ['Date', 'Payee', 'Category', 'Memo', 'Outflow', 'Inflow']
    df_ynab = df_ynab[cols]
    df_ynab['Date'] = df_ynab['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
    
    # process payees
    df_ynab[['Payee', 'Category']] = df_ynab['Payee'].apply(transform_payee).apply(pd.Series)
    return df_ynab

In [34]:
# generate ynab files
for file in result:
    processed_full_path = file['processed_full_path']
    
    print('Generating YNAB file for: [{filename}]'.format(filename=processed_full_path))
    df_in = pd.read_csv(processed_full_path)
    df = process_ynab(df_in)
    file_name = os.path.basename(processed_full_path)
    ynab_df_filename = os.path.splitext(file_name)[0] + '_ynab.csv'
    ynab_df_full_path = os.path.join(ynab_path,ynab_df_filename)
    
    file['ynab_full_path'] = ynab_df_full_path
    
    df.to_csv(ynab_df_full_path, index=False)
    print('YNAB file generated: [{filename}]'.format(filename=ynab_df_full_path))

Generating YNAB file for: [D:\MyDocuments\Bank-Statements\HSBC\2022\3-processed\2022-01-25_Statement_cleaned_processed.csv]
YNAB file generated: [D:\MyDocuments\Bank-Statements\HSBC\2022\4-ynab\2022-01-25_Statement_cleaned_processed_ynab.csv]
Generating YNAB file for: [D:\MyDocuments\Bank-Statements\HSBC\2022\3-processed\2022-02-25_Statement_cleaned_processed.csv]
YNAB file generated: [D:\MyDocuments\Bank-Statements\HSBC\2022\4-ynab\2022-02-25_Statement_cleaned_processed_ynab.csv]
